<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.1: Your First Chisel Module
**Prev: [Introduction to Scala](1_intro_to_scala.ipynb)**<br>
**Next: [Combinational Logic](2.2_comb_logic.ipynb)**

## Motivation
Now that you are familiar with Scala, let's start carving out some hardware! Chisel stands for **C**onstructing **H**ardware **I**n a **S**cala **E**mbedded **L**anguage. That means it is a DSL in Scala, allowing you to take advantage of both Scala and Chisel programming within the same code. It is important to understand which code is "Scala" and which code is "Chisel", but we will discuss that more later. For now, think of Chisel and the code in Module 2 as a better way to write Verilog. This module throws an entire Chisel `Module` and tester at you. Just get the gist of it for now. You'll see plenty more examples later.

## Setup
The following cell downloads the dependencies needed for Chisel. You will see it in all future notebooks. **Run this cell now**.

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/jovyan/source/load-ivy.sc"

As mentioned in the last module, these statements are needed to import Chisel. **Run this cell now** before running any future code blocks.

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

---
# Your First Module
This section will present your first hardware module, a test case, and how to run it. It will contain many things that you will not understand, and that is OK. We want you to take away the broad strokes, so you can continually return to this complete and working example to reinforce what you've learned.

<span style="color:blue">**Example: A Module**</span><br>
Like Verilog, we can declare module definitions in Chisel. The following example is a Chisel `Module`, `Passthrough`, that has one 4-bit input, `in`, and one 4-bit output, `out`. The module combinationally connects `in` and `out`, so `in` drives `out`.

In [3]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

defined class Passthrough

There's a lot here! The following explains how to think of each line in terms of the hardware we are describing.

```scala
class Passthrough extends Module {
```
We declare a new module called `Passthrough`. `Module` is a built-in Chisel class that all hardware modules must extend.

```scala 
val io = IO(...)
```
We declare all our input and output ports in a special `io` `val`. It must be called `io` and be an `IO` object or instance, which requires something of the form `IO(_instantiated_bundle_)`.

```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```
We declare a new hardware struct type (Bundle) that contains some named signals `in` and `out` with directions Input and Output, respectively.

```scala
UInt(4.W)
```
We declare a signal's hardware type. In this case, it is an unsigned integer of width 4.

```scala
io.out := io.in
```
We connect our input port to our output port, such that `io.in` *drives* `io.out`. Note that the `:=` operator is a ***Chisel*** operator that indicates that the right-hand signal drives the left-hand signal. It is a directioned operator.

The neat thing about hardware construction languages (HCLs) is that we can use the underlying programming language as a scripting language. For example, after declaring our Chisel module, we then use Scala to call the Chisel compiler to translate Chisel `Passthrough` into Verilog `Passthrough`. This process is called ***elaboration***.

In [4]:
// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd2.sc 6:10]
endmodule



<span style="color:blue">**Example: A Module Generator**</span><br>
If we apply what we learned about Scala to this example, we can see that a Chisel module is implemented as a Scala class. Just like any other Scala class, we could make a Chisel module take some construction parameters. In this case, we make a new class `PassthroughGenerator` which will accept an integer `width` that dictates the widths of its input and output ports:

In [5]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// Let's now generate modules with different widths
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  output [9:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 6:10]
endmodule

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input         clock,
  input         reset,
  input  [19:0] io_in,
  output [19:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 6:10]
endmodule



defined class PassthroughGenerator

Notice that the generated Verilog uses different bitwidths for the input/output depending on the value assigned to the `width` parameter. Let's dig into how this works. Because Chisel Modules are normal Scala classes, we can use the power of Scala's class constructors to parameterize the elaboration of our design.

You may notice that this parameterization is enabled by *Scala*, not *Chisel*; Chisel has no extra APIs for parameterization, but a designer can simply leverage Scala features to parameterize his/her designs.

Because `PassthroughGenerator` no longer describes a single Module, but instead describes a family of modules parameterized by `width`, we refer to this `Passthrough` as a ***generator***.

---
# Testing Your Hardware

No hardware module or generator should be complete without a tester. Chisel has built-in test features that you will explore throughout this bootcamp. The following example is a Chisel test harness that passes values to an instance of `Passthrough`'s input port `in`, and checks that the same value is seen on the output port `out`.

<span style="color:blue">**Example: A Tester**</span><br>
There is some advanced Scala going on here. However, there is no need for you to understand anything except the `poke` and `expect` commands. You can think of the rest of the code as simply boilerplate to write these simple tests.

In [6]:
// Scala Code: `test` runs the unit test. 
// test takes a user Module and has a code block that applies pokes and expects to the 
// circuit under test (c)
test(new Passthrough()) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


Elaborating design...
Done elaborating.
test Passthrough Success: 0 tests passed in 2 cycles in 0.125647 seconds 15.92 Hz
SUCCESS!!


What's going on? The test accepts a `Passthrough` module, assigns values to the module's inputs, and checks its outputs. To set an input, we call `poke`. To check an output, we call `expect`. If we don't want to compare the output to an expected value (no assertion), we can `peek` the output instead.

If all `expect` statements are true, then our boilerplate code will return pass.

>Note that the `poke` and `expect` use chisel hardware literal notation. Both operations expect literals of the correct type.
If `poke`ing a `UInt()` you must supply a `UInt` literal (example: `c.io.in.poke(10.U)`, likewise if the input is a `Bool()` the `poke` would expect either `true.B` or `false.B`.



<span style="color:red">**Exercise: Writing Your Own Testers**</span><br>
Write and execute two tests, one that tests `PassthroughGenerator` for a width of 10 and a second that tests `PassthroughGenerator` for a width of 20. Check at least two values for each: zero and the maximum value supported by the specified width. Note that the triple question mark has a special meaning in Scala. You may see it frequently in these bootcamp exercises. Running code with the `???` will produce the `NotImplementedError`. Replace `???` with your own code.

In [7]:
// Test with width 10
test(new PassthroughGenerator(10)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(((1 << 10) - 1).U)
    c.io.out.expect(((1 << 10) - 1).U)
}

// Test with width 20

test(new PassthroughGenerator(20)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(((1 << 20) - 1).U)
    c.io.out.expect(((1 << 20) - 1).U)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
test PassthroughGenerator Success: 0 tests passed in 2 cycles in 0.004349 seconds 459.90 Hz
Elaborating design...
Done elaborating.
test PassthroughGenerator Success: 0 tests passed in 2 cycles in 0.005265 seconds 379.86 Hz
SUCCESS!!


<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong> (click to toggle displaying it)</label>
<article>
<pre style="background-color:#f7f7f7">
test(new PassthroughGenerator(10)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1023.U)
    c.io.out.expect(1023.U)
}

test(new PassthroughGenerator(20)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1048575.U)
    c.io.out.expect(1048575.U)
}

</pre></article></div></section></div>

---
# Looking at Generated Verilog/FIRRTL

If you are having trouble understanding the generated hardware and are comfortable with reading structural Verilog and/or FIRRTL (Chisel's IR which is comparable to a synthesis-only subset of Verilog), then you can try looking at the generated Verilog to see the result of Chisel execution.

Here is an example of generating the Verilog (which you've seen already) and the FIRRTL.

In [8]:
// Viewing the Verilog for debugging
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd2.sc 6:10]
endmodule



In [9]:
// Viewing the firrtl for debugging
println(getFirrtl(new Passthrough))

Elaborating design...
Done elaborating.
circuit Passthrough :
  module Passthrough :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<4>, out : UInt<4>}

    io.out <= io.in @[cmd2.sc 6:10]



---
# You're done!

[Return to the top.](#top)

## <span style="color:red"> Appendix: A Note on "printf" Debugging</span>
[Debugging with print statements](https://stackoverflow.com/a/189570) is not always the best way to debug, but is often an easy first step to see what's going on when something doesn't work the way you expect.
Because Chisel generators are programs generating hardware, there are some extra subtleties about printing generator and circuit state.
It is important to remember when your print statement executes and what is being printed.
The three common scenarios where you might want to print have some important differences:
* Chisel generator prints during circuit generation
* Circuit prints during circuit simulation
* Tester prints during testing

`println` is a built-in Scala function that prints to the console. It **cannot** be used to print during circuit simulation because the generated circuit is FIRRTL or Verilog- not Scala.

The following code block shows different styles of printing.

In [10]:
class PrintingModule extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := io.in

    printf("Print during simulation: Input is %d\n", io.in)
    // chisel printf has its own string interpolator too
    printf(p"Print during simulation: IO is $io\n")

    println(s"Print during generation: Input is ${io.in}")
}

test(new PrintingModule ) { c =>
    c.io.in.poke(3.U)
    c.clock.step(5) // circuit will print
    
    println(s"Print during testing: Input is ${c.io.in.peek()}")
}

Elaborating design...
Print during generation: Input is UInt<4>(IO in unelaborated PrintingModule)
Done elaborating.
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during simulation: Input is   3
Print during simulation: IO is AnonymousBundle(in ->   3, out ->   3)
Print during testing: Input is UInt<4>(3)
Print during simulation: Input is   0
Print during simulation: IO is AnonymousBundle(in ->   0, out ->   0)
test PrintingModule Success: 0 tests passed in 7 cycles in 0.073894 seconds 94.73 Hz


defined class PrintingModule

In [11]:
println(getVerilog(new PrintingModule))

Elaborating design...
Print during generation: Input is UInt<4>(IO in unelaborated PrintingModule)
Done elaborating.
module PrintingModule(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  wire  _T_1 = ~reset; // @[cmd9.sc 8:11]
  assign io_out = io_in; // @[cmd9.sc 6:12]
  always @(posedge clock) begin
    `ifndef SYNTHESIS
    `ifdef PRINTF_COND
      if (`PRINTF_COND) begin
    `endif
        if (~reset) begin
          $fwrite(32'h80000002,"Print during simulation: Input is %d\n",io_in); // @[cmd9.sc 8:11]
        end
    `ifdef PRINTF_COND
      end
    `endif
    `endif // SYNTHESIS
    `ifndef SYNTHESIS
    `ifdef PRINTF_COND
      if (`PRINTF_COND) begin
    `endif
        if (_T_1) begin
          $fwrite(32'h80000002,"Print during simulation: IO is AnonymousBundle(in -> %d, out -> %d)\n",io_in,io_out); // @[cmd9.sc 10:11]
        end
    `ifdef PRINTF_COND
      end
    `endif
    `endif // SYNTHESIS
  end
endmodule

